In [1]:
from enum import Enum
from pydantic import BaseModel, Field


class FooBar(BaseModel):
    count: int
    size: float = None


class Gender(str, Enum):
    male = 'male'
    female = 'female'
    other = 'other'
    not_given = 'not_given'


class MainModel(BaseModel):
    """
    This is the description of the main model
    """

    foo_bar: FooBar = Field(...)
    gender: Gender = Field(None, alias='Gender')
    snap: int = Field(
        42,
        title='The Snap',
        description='this is the value of snap',
        gt=30,
        lt=50,
    )

    class Config:
        title = 'Main'


# this is equivalent to json.dumps(MainModel.schema(), indent=2):
print(MainModel.schema_json(indent=2))

{
  "title": "Main",
  "description": "This is the description of the main model",
  "type": "object",
  "properties": {
    "foo_bar": {
      "$ref": "#/definitions/FooBar"
    },
    "Gender": {
      "$ref": "#/definitions/Gender"
    },
    "snap": {
      "title": "The Snap",
      "description": "this is the value of snap",
      "default": 42,
      "exclusiveMinimum": 30,
      "exclusiveMaximum": 50,
      "type": "integer"
    }
  },
  "required": [
    "foo_bar"
  ],
  "definitions": {
    "FooBar": {
      "title": "FooBar",
      "type": "object",
      "properties": {
        "count": {
          "title": "Count",
          "type": "integer"
        },
        "size": {
          "title": "Size",
          "type": "number"
        }
      },
      "required": [
        "count"
      ]
    },
    "Gender": {
      "title": "Gender",
      "description": "An enumeration.",
      "enum": [
        "male",
        "female",
        "other",
        "not_given"
      ],
     

In [2]:
from typing import Literal, Union

from typing_extensions import Annotated

from pydantic import BaseModel, Field, schema_json_of


class Cat(BaseModel):
    pet_type: Literal['cat']
    cat_name: str


class Dog(BaseModel):
    pet_type: Literal['dog']
    dog_name: str


Pet = Annotated[Union[Cat, Dog], Field(discriminator='pet_type')]

print(schema_json_of(Pet, title='The Pet Schema', indent=2))
"""
{
  "title": "The Pet Schema",
  "discriminator": {
    "propertyName": "pet_type",
    "mapping": {
      "cat": "#/definitions/Cat",
      "dog": "#/definitions/Dog"
    }
  },
  "oneOf": [
    {
      "$ref": "#/definitions/Cat"
    },
    {
      "$ref": "#/definitions/Dog"
    }
  ],
  "definitions": {
    "Cat": {
      "title": "Cat",
      "type": "object",
      "properties": {
        "pet_type": {
          "title": "Pet Type",
          "enum": [
            "cat"
          ],
          "type": "string"
        },
        "cat_name": {
          "title": "Cat Name",
          "type": "string"
        }
      },
      "required": [
        "pet_type",
        "cat_name"
      ]
    },
    "Dog": {
      "title": "Dog",
      "type": "object",
      "properties": {
        "pet_type": {
          "title": "Pet Type",
          "enum": [
            "dog"
          ],
          "type": "string"
        },
        "dog_name": {
          "title": "Dog Name",
          "type": "string"
        }
      },
      "required": [
        "pet_type",
        "dog_name"
      ]
    }
  }
}
"""

{
  "title": "The Pet Schema",
  "discriminator": {
    "propertyName": "pet_type",
    "mapping": {
      "cat": "#/definitions/Cat",
      "dog": "#/definitions/Dog"
    }
  },
  "oneOf": [
    {
      "$ref": "#/definitions/Cat"
    },
    {
      "$ref": "#/definitions/Dog"
    }
  ],
  "definitions": {
    "Cat": {
      "title": "Cat",
      "type": "object",
      "properties": {
        "pet_type": {
          "title": "Pet Type",
          "enum": [
            "cat"
          ],
          "type": "string"
        },
        "cat_name": {
          "title": "Cat Name",
          "type": "string"
        }
      },
      "required": [
        "pet_type",
        "cat_name"
      ]
    },
    "Dog": {
      "title": "Dog",
      "type": "object",
      "properties": {
        "pet_type": {
          "title": "Pet Type",
          "enum": [
            "dog"
          ],
          "type": "string"
        },
        "dog_name": {
          "title": "Dog Name",
          "type": "

'\n{\n  "title": "The Pet Schema",\n  "discriminator": {\n    "propertyName": "pet_type",\n    "mapping": {\n      "cat": "#/definitions/Cat",\n      "dog": "#/definitions/Dog"\n    }\n  },\n  "oneOf": [\n    {\n      "$ref": "#/definitions/Cat"\n    },\n    {\n      "$ref": "#/definitions/Dog"\n    }\n  ],\n  "definitions": {\n    "Cat": {\n      "title": "Cat",\n      "type": "object",\n      "properties": {\n        "pet_type": {\n          "title": "Pet Type",\n          "enum": [\n            "cat"\n          ],\n          "type": "string"\n        },\n        "cat_name": {\n          "title": "Cat Name",\n          "type": "string"\n        }\n      },\n      "required": [\n        "pet_type",\n        "cat_name"\n      ]\n    },\n    "Dog": {\n      "title": "Dog",\n      "type": "object",\n      "properties": {\n        "pet_type": {\n          "title": "Pet Type",\n          "enum": [\n            "dog"\n          ],\n          "type": "string"\n        },\n        "dog_name":

In [3]:
from pydantic import BaseModel, Field, PositiveInt

try:
    # this won't work since PositiveInt takes precedence over the
    # constraints defined in Field meaning they're ignored
    class Model(BaseModel):
        foo: PositiveInt = Field(..., lt=10)
except ValueError as e:
    print(e)
    """
    On field "foo" the following field constraints are set but not enforced:
    lt.
    For more details see https://docs.pydantic.dev/usage/schema/#unenforced-
    field-constraints
    """


# but you can set the schema attribute directly:
# (Note: here exclusiveMaximum will not be enforce)
class Model(BaseModel):
    foo: PositiveInt = Field(..., exclusiveMaximum=10)


print(Model.schema())
"""
{
    'title': 'Model',
    'type': 'object',
    'properties': {
        'foo': {
            'title': 'Foo',
            'exclusiveMaximum': 10,
            'exclusiveMinimum': 0,
            'type': 'integer',
        },
    },
    'required': ['foo'],
}
"""


# if you find yourself needing this, an alternative is to declare
# the constraints in Field (or you could use conint())
# here both constraints will be enforced:
class Model(BaseModel):
    # Here both constraints will be applied and the schema
    # will be generated correctly
    foo: int = Field(..., gt=0, lt=10)


print(Model.schema())
"""
{
    'title': 'Model',
    'type': 'object',
    'properties': {
        'foo': {
            'title': 'Foo',
            'exclusiveMinimum': 0,
            'exclusiveMaximum': 10,
            'type': 'integer',
        },
    },
    'required': ['foo'],
}
"""

On field "foo" the following field constraints are set but not enforced: lt. 
For more details see https://pydantic-docs.helpmanual.io/usage/schema/#unenforced-field-constraints
{'title': 'Model', 'type': 'object', 'properties': {'foo': {'title': 'Foo', 'exclusiveMaximum': 10, 'exclusiveMinimum': 0, 'type': 'integer'}}, 'required': ['foo']}
{'title': 'Model', 'type': 'object', 'properties': {'foo': {'title': 'Foo', 'exclusiveMinimum': 0, 'exclusiveMaximum': 10, 'type': 'integer'}}, 'required': ['foo']}


"\n{\n    'title': 'Model',\n    'type': 'object',\n    'properties': {\n        'foo': {\n            'title': 'Foo',\n            'exclusiveMinimum': 0,\n            'exclusiveMaximum': 10,\n            'type': 'integer',\n        },\n    },\n    'required': ['foo'],\n}\n"

In [4]:
from uuid import uuid4

from pydantic import BaseModel, Field
from typing_extensions import Annotated


class Foo(BaseModel):
    id: Annotated[str, Field(default_factory=lambda: uuid4().hex)]
    name: Annotated[str, Field(max_length=256)] = 'Bar'

In [5]:
from typing import Any, Callable, Dict, Generator, Optional

from pydantic import BaseModel, Field
from pydantic.fields import ModelField


class RestrictedAlphabetStr(str):
    @classmethod
    def __get_validators__(cls) -> Generator[Callable, None, None]:
        yield cls.validate

    @classmethod
    def validate(cls, value: str, field: ModelField):
        alphabet = field.field_info.extra['alphabet']
        if any(c not in alphabet for c in value):
            raise ValueError(f'{value!r} is not restricted to {alphabet!r}')
        return cls(value)

    @classmethod
    def __modify_schema__(
        cls, field_schema: Dict[str, Any], field: Optional[ModelField]
    ):
        if field:
            alphabet = field.field_info.extra['alphabet']
            field_schema['examples'] = [c * 3 for c in alphabet]


class MyModel(BaseModel):
    value: RestrictedAlphabetStr = Field(alphabet='ABC')


print(MyModel.schema_json(indent=2))

{
  "title": "MyModel",
  "type": "object",
  "properties": {
    "value": {
      "title": "Value",
      "alphabet": "ABC",
      "examples": [
        "AAA",
        "BBB",
        "CCC"
      ],
      "type": "string"
    }
  },
  "required": [
    "value"
  ]
}


In [6]:
import json
from pydantic import BaseModel
from pydantic.schema import schema


class Foo(BaseModel):
    a: str = None


class Model(BaseModel):
    b: Foo


class Bar(BaseModel):
    c: int


top_level_schema = schema([Model, Bar], title='My Schema')
print(json.dumps(top_level_schema, indent=2))

{
  "title": "My Schema",
  "definitions": {
    "Foo": {
      "title": "Foo",
      "type": "object",
      "properties": {
        "a": {
          "title": "A",
          "type": "string"
        }
      }
    },
    "Model": {
      "title": "Model",
      "type": "object",
      "properties": {
        "b": {
          "$ref": "#/definitions/Foo"
        }
      },
      "required": [
        "b"
      ]
    },
    "Bar": {
      "title": "Bar",
      "type": "object",
      "properties": {
        "c": {
          "title": "C",
          "type": "integer"
        }
      },
      "required": [
        "c"
      ]
    }
  }
}


In [7]:
import json
from pydantic import BaseModel
from pydantic.schema import schema


class Foo(BaseModel):
    a: int


class Model(BaseModel):
    a: Foo


# Default location for OpenAPI
top_level_schema = schema([Model], ref_prefix='#/components/schemas/')
print(json.dumps(top_level_schema, indent=2))

{
  "definitions": {
    "Foo": {
      "title": "Foo",
      "type": "object",
      "properties": {
        "a": {
          "title": "A",
          "type": "integer"
        }
      },
      "required": [
        "a"
      ]
    },
    "Model": {
      "title": "Model",
      "type": "object",
      "properties": {
        "a": {
          "$ref": "#/components/schemas/Foo"
        }
      },
      "required": [
        "a"
      ]
    }
  }
}


In [8]:
from pydantic import BaseModel


class Person(BaseModel):
    name: str
    age: int

    class Config:
        schema_extra = {
            'examples': [
                {
                    'name': 'John Doe',
                    'age': 25,
                }
            ]
        }


print(Person.schema_json(indent=2))

{
  "title": "Person",
  "type": "object",
  "properties": {
    "name": {
      "title": "Name",
      "type": "string"
    },
    "age": {
      "title": "Age",
      "type": "integer"
    }
  },
  "required": [
    "name",
    "age"
  ],
  "examples": [
    {
      "name": "John Doe",
      "age": 25
    }
  ]
}


In [9]:
from typing import Dict, Any, Type
from pydantic import BaseModel


class Person(BaseModel):
    name: str
    age: int

    class Config:
        @staticmethod
        def schema_extra(schema: Dict[str, Any], model: Type['Person']) -> None:
            for prop in schema.get('properties', {}).values():
                prop.pop('title', None)


print(Person.schema_json(indent=2))

{
  "title": "Person",
  "type": "object",
  "properties": {
    "name": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    }
  },
  "required": [
    "name",
    "age"
  ]
}
